### Lesson 3

In [ ]:
import os
import sys
import langchain
import subprocess
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from Update_Git import git_add, git_commit, git_push
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_dir, ".env")
load_dotenv(dotenv_path=env_path)

add_file = os.path.join(current_dir, 'Lesson_3.ipynb')
git_add(add_file)
git_commit('Lesson 2 updated')
git_push('main')

Error executing command: ['git', 'add', 'path_to_your_file']
Error message: fatal: pathspec 'path_to_your_file' did not match any files

Failed to add path_to_your_file to git.
